# MODELING DATAFRAME CREATION

In [1]:
#Load dependencies
from datetime import datetime as dt, timedelta
import pandas as pd
import numpy as np
from uszipcode import SearchEngine

#Open case close date
opencaseclosedate=np.datetime64('2021-01-01')

In [2]:
#Define data files to import
url2017='https://hfdapp.houstontx.gov/311/311-Public-Data-Extract-2017.txt'
url2018='https://hfdapp.houstontx.gov/311/311-Public-Data-Extract-2018.txt'
url2019='https://hfdapp.houstontx.gov/311/311-Public-Data-Extract-2019.txt'
url2020='https://hfdapp.houstontx.gov/311/311-Public-Data-Extract-monthly.txt'
nullzip=pd.read_csv('../Clean Data Files/311latlngzipcodes.csv',dtype={'calczip':str})

#Define dataframe column names and select numeric and date columns
cols=['case number','sr location','county','district','neighborhood','tax id','trash quad','recycle quad','trash day','heavy trash day','recycle day','key map',
      'management district','department','division','sr type','queue','sla','status','sr create date','due date','date closed','overdue','title','x','y','latitude',
      'longitude','channel type']
numcols=['latitude','longitude']
datecols=['sr create date','due date','date closed']

#Create zipcode retrieval function
search=SearchEngine(simple_zipcode=False)
def zipinfo(lat,lng):
    zipdata=search.by_coordinates(lat,lng,radius=3,returns=1)
    for zipcode in zipdata:
        return zipcode.zipcode

In [3]:
#Create 2017 data frames
data2017=pd.read_csv(url2017,header=5,sep='|',error_bad_lines=False)
data2017=data2017.drop(data2017.index[0]).reset_index(drop=True)
data2017.columns=cols
data2017[cols]=data2017[cols].apply(lambda x:x.str.strip()).replace(r'^\s*$',np.nan,regex=True)
data2017[datecols]=data2017[datecols].apply(pd.to_datetime,format='%Y-%m-%d %H:%M:%S',errors='coerce')
data2017[numcols]=data2017[numcols].apply(pd.to_numeric,errors='coerce')
top2017=data2017['sr type'].value_counts()[lambda x:x>=10000].index.tolist()
data2017.shape

b'Skipping line 9979: expected 29 fields, saw 30\nSkipping line 16339: expected 29 fields, saw 30\n'
b'Skipping line 211068: expected 29 fields, saw 30\n'
b'Skipping line 294299: expected 29 fields, saw 30\n'
b'Skipping line 327924: expected 29 fields, saw 30\n'


(364664, 29)

In [4]:
#Create 2018 data frames
data2018=pd.read_csv(url2018,header=5,sep='|',error_bad_lines=False)
data2018=data2018.drop(data2018.index[0]).reset_index(drop=True)
data2018.columns=cols
data2018[cols]=data2018[cols].apply(lambda x:x.str.strip()).replace(r'^\s*$',np.nan,regex=True)
data2018[datecols]=data2018[datecols].apply(pd.to_datetime,format='%Y-%m-%d %H:%M:%S',errors='coerce')
data2018[numcols]=data2018[numcols].apply(pd.to_numeric,errors='coerce')
top2018=data2018['sr type'].value_counts()[lambda x:x>=10000].index.tolist()
data2018.shape

b'Skipping line 124864: expected 29 fields, saw 30\n'
C:\Users\phili\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(399953, 29)

In [5]:
#Create 2019 data frames
data2019=pd.read_csv(url2019,header=5,sep='|',error_bad_lines=False)
data2019=data2019.drop(data2019.index[0]).reset_index(drop=True)
data2019.columns=cols
data2019[cols]=data2019[cols].apply(lambda x:x.str.strip()).replace(r'^\s*$',np.nan,regex=True)
data2019[datecols]=data2019[datecols].apply(pd.to_datetime,format='%Y-%m-%d %H:%M:%S',errors='coerce')
data2019[numcols]=data2019[numcols].apply(pd.to_numeric,errors='coerce')
top2019=data2019['sr type'].value_counts()[lambda x:x>=10000].index.tolist()
data2019.shape

b'Skipping line 86859: expected 29 fields, saw 31\n'
b'Skipping line 124913: expected 29 fields, saw 30\n'
b'Skipping line 144497: expected 29 fields, saw 30\n'
b'Skipping line 218652: expected 29 fields, saw 31\n'
b'Skipping line 349873: expected 29 fields, saw 30\n'


(395258, 29)

In [6]:
#Create all complete years dataframe
tempdata311=data2017.append([data2018,data2019])
data311=pd.merge(tempdata311,nullzip,on=['latitude','longitude'],how='left')
data311['date']=data311['sr create date'].dt.strftime('%Y-%m-%d')
data311['year']=data311['sr create date'].dt.strftime('%Y')
data311['month']=data311['sr create date'].dt.strftime('%m')
data311=data311[pd.notnull(data311['latitude'])]
data311['date closed']=np.where(data311['date closed'].isnull()==True,opencaseclosedate,data311['date closed'])
data311['truezip']='77'+data311['sr location'].str.extract(r'77(\d{3}\-?\d{0,4})')
data311['zipcode']=np.where(data311['truezip'].isnull()==True,data311['calczip'],data311['truezip'])
data311['openclosetime']=data311['date closed']-data311['sr create date']
data311['daystoclose']=data311['openclosetime']/timedelta(days=1)
data311['openduetime']=data311['due date']-data311['sr create date']
data311['daysdue']=data311['openduetime']/timedelta(days=1)
data311['missedduedate']=np.where(data311['due date']>data311['date closed'],0,1)
data311.drop(['x','y','calczip','truezip','openclosetime','openduetime'],axis=1,inplace=True)
types311=data311.groupby(['sr type','year'])['case number'].count().unstack('year').reset_index()
types311.columns=['sr type','2017','2018','2019']
data311.shape

(1060028, 34)

In [7]:
#Create dataframe containing service requests with around 10000 a year
toprequests=sorted(np.unique(top2017+top2018+top2019))
topdata=data311.loc[data311['sr type'].isin(toprequests)].reset_index()
topdata.drop(['index'],axis=1,inplace=True)

In [8]:
#Create weather and census dataframes
weatherdata=pd.read_csv('../static/data/Final_selected_weather_data.csv')
censusdata=pd.read_csv('../Clean Data Files/census_data.csv')

In [9]:
#Apply a few fixes to weather and census dataframes for merging
weatherdata.drop(['Unnamed: 0'],axis=1,inplace=True)
censusdata['zipcode']=censusdata['Zipcode'].astype(str)
censusdata.drop(['Zipcode'],axis=1,inplace=True)

In [10]:
#Merge dataframes to create modeling dataframe
tempmodeldata=pd.merge(topdata,weatherdata,how='left',left_on='date',right_on='date_field')
modeldata=pd.merge(tempmodeldata,censusdata,how='left',on='zipcode') 

In [11]:
#Display null values in dataset
modeldata.isnull().sum()

case number                  0
sr location                  0
county                    1968
district                  2798
neighborhood              3755
tax id                    1958
trash quad              106574
recycle quad            108723
trash day               106574
heavy trash day         107184
recycle day             108721
key map                      0
management district     380783
department                   0
division                     0
sr type                      0
queue                        0
sla                          0
status                       0
sr create date               0
due date                     6
date closed                  0
overdue                   4982
title                        0
latitude                     0
longitude                    0
channel type                 0
date                         0
year                         0
month                        0
zipcode                   6834
daystoclose                  0
daysdue 

In [12]:
#Create modeling data csv and json files
#modeldata.to_csv('../Clean Data Files/modeldata.csv',index=False,header=True)
#modeldata.to_json('../static/data/modeldata.json',orient='records')

In [13]:
modeldata.shape

(657207, 47)

In [14]:
modeldata.columns

Index(['case number', 'sr location', 'county', 'district', 'neighborhood',
       'tax id', 'trash quad', 'recycle quad', 'trash day', 'heavy trash day',
       'recycle day', 'key map', 'management district', 'department',
       'division', 'sr type', 'queue', 'sla', 'status', 'sr create date',
       'due date', 'date closed', 'overdue', 'title', 'latitude', 'longitude',
       'channel type', 'date', 'year', 'month', 'zipcode', 'daystoclose',
       'daysdue', 'missedduedate', 'date_field', 'tempMax', 'tempAvg',
       'tempMin', 'precipitation', 'Population', 'Median Age',
       'Household Income', 'Per Capita Income', 'Poverty Rate',
       'Total Households', 'Total Owner Occupied', '% Owner Occupied'],
      dtype='object')

In [15]:
modeldata

,case number,sr location,county,district,neighborhood,tax id,trash quad,recycle quad,trash day,heavy trash day,...,tempMin,precipitation,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Total Households,Total Owner Occupied,% Owner Occupied
0,101002444726,Intersection 3900 S GESSNER RD&10000 WESTPARK DR,Harris County,F,MID WEST,NULL,NaN,NaN,NaN,NaN,...,64,0.3,38931.0,31.9,44957.0,32805.0,22.824998,38751.0,9667.0,24.946453
1,12091836-101002444730,"3303 SAGE, HOUSTON TX 77056",HARRIS,G,GREATER UPTOWN,0451400060009,NaN,NaN,NaN,NaN,...,64,0.3,21732.0,39.1,107003.0,89180.0,5.268728,21641.0,10643.0,49.179798
2,101002444733,Intersection 1400 CAROLINE ST&1300 CLAY ST,Harris County,I,DOWNTOWN,NULL,NaN,NaN,NaN,NaN,...,64,0.3,915.0,44.6,250001.0,196722.0,6.666667,915.0,78.0,8.524590
3,12091839-101002444736,"7701 APPLETON, HOUSTON TX 77022",HARRIS,H,NORTHSIDE/NORTHLINE,0710210010015,NE,NW,MONDAY,3rd Monday,...,64,0.3,27364.0,34.1,30164.0,14924.0,29.399942,27186.0,13143.0,48.344736
4,12091840-101002444737,"7701 APPLETON, HOUSTON TX 77022",HARRIS,H,NORTHSIDE/NORTHLINE,0420050000055,NE,NW,MONDAY,3rd Monday,...,64,0.3,27364.0,34.1,30164.0,14924.0,29.399942,27186.0,13143.0,48.344736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657202,101003785782,"3614 S MACGREGOR, HOUSTON TX 77021",HARRIS,D,MACGREGOR,0611350550011,SE,SW,TUESDAY,3rd Thursday,...,38,0.0,26654.0,33.6,35126.0,21531.0,28.674871,26327.0,10572.0,40.156493
657203,101003785783,"3547 TAMPA, HOUSTON TX 77021",HARRIS,D,MACGREGOR,0741210010025,SE,SW,TUESDAY,3rd Thursday,...,38,0.0,26654.0,33.6,35126.0,21531.0,28.674871,26327.0,10572.0,40.156493
657204,101003785785,"3415 WENTWORTH, HOUSTON TX 77004",HARRIS,D,MACGREGOR,0700280060004,SE,SW,TUESDAY,3rd Friday,...,38,0.0,37642.0,28.3,48592.0,31067.0,19.733277,28125.0,9997.0,35.544889
657205,20024521-101003785786,"11035 AVON BROOK, HOUSTON TX 77034",HARRIS,E,SOUTH BELT / ELLINGTON,1319170030001,SE,SE,THURSDAY,2nd Thursday,...,38,0.0,40183.0,28.1,47252.0,19317.0,19.767066,40162.0,19810.0,49.325233


# MODELING

In [18]:
nulldata=pd.isnull(topdata['zipcode'])
topdata[nulldata]

,case number,sr location,county,district,neighborhood,tax id,trash quad,recycle quad,trash day,heavy trash day,...,latitude,longitude,channel type,date,year,month,zipcode,daystoclose,daysdue,missedduedate
272,12092050-101002445261,Intersection 1600 SABINE ST&1800 WINTER ST,Harris County,H,WASHINGTON AVENUE COALITION / MEMORIAL P,NULL,NW,NW,MONDAY,1st Wednesday,...,29.772294,-95.375870,Voice In,2017-01-02,2017,01,NaN,3.046632,29.999988,0
326,1330046-101002445364,Intersection 6600 FOXFERN CIR&13512 WILEYWOOD DR,Harris County,I,HUNTERWOOD,NULL,NE,NE,THURSDAY,1st Monday,...,29.821708,-95.207428,Voice In,2017-01-02,2017,01,NaN,3.740937,12.999988,0
350,101002445409,Intersection 12600 BRIAR FOREST DR&1420 S DAIR...,Harris County,G,ELDRIDGE / WEST OAKS,NULL,NaN,NaN,NaN,NaN,...,29.753922,-95.606110,Voice In,2017-01-02,2017,01,NaN,0.096481,1.000000,0
392,12092139-101002445484,Intersection 2300 BARKER CYPRESS RD&18000 SAUM...,Harris County,A,ADDICKS PARK TEN,NULL,NaN,NaN,NaN,NaN,...,29.800333,-95.688469,Voice In,2017-01-02,2017,01,NaN,0.589699,9.998692,0
636,12092308-101002446022,Intersection 2100 SAN FELIPE ST&1900 S SHEPHER...,Harris County,G,NEARTOWN - MONTROSE,NULL,NaN,SW,NaN,NaN,...,29.749075,-95.410667,Voice In,2017-01-03,2017,01,NaN,7.935914,30.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626831,1682528-101003697806,Intersection 9600 CIBOLO&9626 SALTILLO ST,Harris County,I,EL DORADO / OATES PRAIRIE,NULL,NE,NE,THURSDAY,1st Monday,...,29.800013,-95.257828,Voice In,2019-10-25,2019,10,NaN,11.060104,20.000000,0
627115,1682718-101003698393,Intersection 11132 BRAES FOREST DR&7900 DUFFIE...,Harris County,K,BRAYS OAKS,NULL,SW,SW,THURSDAY,2nd Friday,...,29.656876,-95.520454,Voice In,2019-10-25,2019,10,NaN,9.671655,19.589421,0
627501,1682783-101003699137,Intersection 7300 CAYTON ST&6930 PRENTISS DR,Harris County,I,GOLFCREST / BELLFORT / REVEILLE,NULL,SE,SE,FRIDAY,3rd Monday,...,29.659113,-95.297226,Voice In,2019-10-27,2019,10,NaN,4.998229,18.860440,0
627622,1682806-101003699384,Intersection 700 FAIRVIEW AVE&2400 STANFORD ST,Harris County,C,NEARTOWN - MONTROSE,NULL,NW,NE,MONDAY,1st Wednesday,...,29.748138,-95.388873,Voice In,2019-10-28,2019,10,NaN,1.214977,18.302836,0
